In [1]:
import os
import sys
from datetime import *
import time
from urllib.request import urlretrieve
from urllib.error import URLError
from socket import timeout

In [2]:
resultDirectory = 'dataset/hycom/'
date_start = '2019-01-01 12:00:00'      
number_of_days  = 1
glb = 'GLBy0.08'                     
expt = '93.0'

north = -2.438228            # -80 to 80 degN          
south = -7.514981            # -80 to 80 degN
west = 118.414307        # 0 to 360 degE
east = 124.533692        # 0 to 360 degE

var_list = 'surf_el,water_temp,salinity,water_u,water_v'

In [3]:
def ensure_dir(file_path):
    # create a folder if it does not already exist
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [4]:
def get_extraction(dt, out_fn, var_list):
    dstr0 = dt.strftime('%Y-%m-%d-T%H:00:00Z')
    print(dstr0)
    if expt == '53.X':
        url = ('http://ncss.hycom.org/thredds/ncss/' + glb + '/expt_' + expt + '/data/' + dt.strftime('%Y') + 
            '?var='+var_list +
            '&north='+str(north)+'&south='+str(south)+'&west='+str(west)+'&east='+str(east) +
            '&disableProjSubset=on&horizStride=1' +
            '&time_start='+dstr0+'&time_end='+dstr0+'&timeStride=8' +
            '&vertCoord=&addLatLon=true&accept=netcdf4')
    else:
        url = ('http://ncss.hycom.org/thredds/ncss/' + glb + '/expt_' + expt + 
            '?var='+var_list +
            '&north='+str(north)+'&south='+str(south)+'&west='+str(west)+'&east='+str(east) +
            '&disableProjSubset=on&horizStride=1' +
            '&time_start='+dstr0+'&time_end='+dstr0+'&timeStride=8' +
            '&vertCoord=&addLatLon=true&accept=netcdf4')
    print(url)
    # get the data and save as a netcdf file
    counter = 1
    got_file = False
    while (counter <= 10) and (got_file == False):
        print('  Attempting to get data, counter = ' + str(counter))
        tt0 = time.time()
        try:
            (a,b) = urlretrieve(url, out_fn)
            # a is the output file name
            # b is a message you can see with b.as_string()
        except URLError as ee:
            if hasattr(ee, 'reason'):
                print('  *We failed to reach a server.')
                print('  -Reason: ', ee.reason)
            elif hasattr(ee, 'code'):
                print('  *The server could not fulfill the request.')
                print('  -Error code: ', ee.code)
        except timeout:
            print('  *Socket timed out')
        else:
            got_file = True
            print('  Downloaded data')
        print('  Time elapsed: %0.1f seconds' % (time.time() - tt0))
        counter += 1
    if got_file:
        result = 'success'
    else:
        result = 'fail'
    return result

In [5]:
base = datetime.fromisoformat(date_start)
if base.strftime('%H') == '12' and number_of_days >= 365:
    ndt = number_of_days * 8 - 4
else:
    ndt = number_of_days * 8
dt_list = []
dt_list = [base + timedelta(hours=3*x) for x in range(ndt)]

In [6]:

out_dir = resultDirectory                  # specify output directory adding the ending '/'
ensure_dir(out_dir)                        # make sure the output directory exists, make one if not
f = open(out_dir + 'log.txt', 'w+')        # open log of successful downloads
print('\n** Working on ' + glb + '/expt_' + expt + ' **')
f.write('\n\n** Working on ' + glb + '/expt_' + expt + ' **')
tt1 = time.time()                          # tic for total elapsed time
force_overwrite = True                     # overwrite any already existing nc files in the output folder
for dt in dt_list:
    out_fn = out_dir + datetime.strftime(dt, '%Y%m%d_%H') + '.nc'
    print(out_fn)
    if os.path.isfile(out_fn):
        if force_overwrite:
            os.remove(out_fn)
    if not os.path.isfile(out_fn):
        result = get_extraction(dt, out_fn, var_list)
        f.write('\n ' + datetime.strftime(dt, '%Y%m%d_%H') + ' ' + result)

totmin = (time.time() - tt1)/60             # total time elapsed for loop over all datetimes in minutes
print('')
print('All downloads are completed.')
print('Total time elapsed: %0.1f minutes' % totmin)
f.close()       # close log of successful downloads


** Working on GLBy0.08/expt_93.0 **
dataset/hycom/20190101_12.nc
2019-01-01-T12:00:00Z
http://ncss.hycom.org/thredds/ncss/GLBy0.08/expt_93.0?var=surf_el,water_temp,salinity,water_u,water_v&north=-2.438228&south=-7.514981&west=118.414307&east=124.533692&disableProjSubset=on&horizStride=1&time_start=2019-01-01-T12:00:00Z&time_end=2019-01-01-T12:00:00Z&timeStride=8&vertCoord=&addLatLon=true&accept=netcdf4
  Attempting to get data, counter = 1
  Downloaded data
  Time elapsed: 32.0 seconds
dataset/hycom/20190101_15.nc
2019-01-01-T15:00:00Z
http://ncss.hycom.org/thredds/ncss/GLBy0.08/expt_93.0?var=surf_el,water_temp,salinity,water_u,water_v&north=-2.438228&south=-7.514981&west=118.414307&east=124.533692&disableProjSubset=on&horizStride=1&time_start=2019-01-01-T15:00:00Z&time_end=2019-01-01-T15:00:00Z&timeStride=8&vertCoord=&addLatLon=true&accept=netcdf4
  Attempting to get data, counter = 1
  Downloaded data
  Time elapsed: 65.3 seconds
dataset/hycom/20190101_18.nc
2019-01-01-T18:00:00Z
ht